In [1]:
from openai import OpenAI
import pandas as pd

In [5]:
# Example domain and format for the MySQL table
domain = "Healthcare patient records"
columns_format = {
    "PatientID": "int",
    "Name": "str",
    "Age": "int",
    "Diagnosis": "str",
    "LastVisitDate": "datetime"
}

In [6]:
# Your OpenAI API key
client = OpenAI(api_key=api_key)

In [15]:
def generate_synthetic_data_with_chat_model(domain, columns_format, n_records=10):
    try:
        response = client.chat.completions.create(model="gpt-3.5-turbo",  # Ensure this is a chat model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Generate {n_records} synthetic records for {domain} with the following format: {list(columns_format.keys())}. This will be used in a MySQL database."}
        ])
        # Extracting and returning the generated text
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [16]:
# Example usage
# Define the domain and columns format for the synthetic data generation
domain = "healthcare"
columns_format = {
    "First Name": "varchar(20)",
    "Last Name": "varchar(20)",
    "Age": "int(3)", 
    "DOB": "datetime", 
    "Diagnosis": "varchar(200)",
    "Gender": "boolean",
    "BloodType": "varchar(3)",
    "Allergies": "varchar(200)",
    "LastVisitDate": "datetime",
    "Medications": "varchar(200)"
}

# Generate synthetic data (ensure to replace 'your_openai_api_key_here' with your actual API key)
synthetic_data = generate_synthetic_data_with_chat_model(domain, columns_format, 10)

# Optionally, print or further process the synthetic data
print(synthetic_data)


Sure! Here are 10 synthetic records for healthcare in the specified format:

1. ['Alice', 'Smith', 45, '1976-03-12', 'Hypertension', 'Female', 'A+', 'None', '2021-09-20', 'Lisinopril']
2. ['John', 'Doe', 30, '1991-07-25', 'Diabetes', 'Male', 'O-', 'Peanuts', '2021-08-15', 'Metformin']
3. ['Emily', 'Johnson', 55, '1966-11-04', 'Arthritis', 'Female', 'AB+', 'Penicillin', '2021-10-10', 'Ibuprofen']
4. ['David', 'Williams', 68, '1953-05-19', 'Heart Disease', 'Male', 'B-', 'Shellfish', '2021-07-30', 'Atorvastatin']
5. ['Sarah', 'Brown', 25, '1996-09-08', 'Asthma', 'Female', 'A-', 'Dust', '2021-11-05', 'Albuterol']
6. ['Michael', 'Jones', 40, '1981-02-10', 'Depression', 'Male', 'A+', 'Cats', '2021-09-25', 'Sertraline']
7. ['Laura', 'Martinez', 35, '1986-04-23', 'Migraine', 'Female', 'O+', 'None', '2021-10-30', 'Sumatriptan']
8. ['Daniel', 'Garcia', 50, '1971-08-15', 'High Cholesterol', 'Male', 'AB-', 'Pollen', '2021-08-28', 'Rosuvastatin']
9. ['Jessica', 'Lee', 60, '1961-12-30', 'Osteoporosi

In [14]:
synthetic_data

"Sure! Here are 10 synthetic records for healthcare:\n\n1. ['Emily Johnson', 45, 'Hypertension']\n2. ['David Smith', 33, 'Diabetes']\n3. ['Samantha Brown', 28, 'Migraine']\n4. ['Michael Rodriguez', 50, 'Arthritis']\n5. ['Jennifer Lee', 60, 'Osteoporosis']\n6. ['Brian Williams', 42, 'Anxiety']\n7. ['Laura Davis', 55, 'Cancer']\n8. ['Mark Thompson', 37, 'Asthma']\n9. ['Karen Martinez', 70, 'Alzheimer's Disease']\n10. ['Daniel Wilson', 25, 'Depression']"

In [12]:
# Function to convert generated data to pandas DataFrame
def convert_to_dataframe(raw_data, columns_format):
    # Split the raw data into lines and then into fields
    lines = raw_data.split('\n')
    data = [line.split(',') for line in lines if line.strip() != '']  # Assuming comma-separated values
    
    # Convert to DataFrame
    df = pd.DataFrame(data, columns=columns_format.keys())
    
    # Convert types
    for column, dtype in columns_format.items():
        if dtype == 'int':
            df[column] = pd.to_numeric(df[column], errors='coerce').fillna(0).astype(int)
        elif dtype == 'datetime':
            df[column] = pd.to_datetime(df[column], errors='coerce')
        # Add more type conversions as needed
    
    return df

In [13]:
# Convert to DataFrame and adjust types
df = convert_to_dataframe(synthetic_data, columns_format)

# Save to CSV
df.to_csv('synthetic_healthcare_data.csv', index=False)

print("Synthetic data generated and saved to synthetic_healthcare_data.csv.")


Synthetic data generated and saved to synthetic_healthcare_data.csv.


In [4]:
# from openai import OpenAI
# client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [17]:
def parse_and_convert_to_sql(generated_data, table_name="HealthcareRecords"):
    """
    Parses generated data and converts it into SQL INSERT statements.

    Parameters:
    - generated_data (str): The synthetic data generated by the model.
    - table_name (str): The name of the database table for the INSERT statements.

    Returns:
    - str: A string containing SQL INSERT statements for the generated data.
    """
    # Assuming generated_data is a newline-separated list of records
    records = generated_data.split('\n')
    
    # The base SQL INSERT statement format
    sql_insert_template = f"INSERT INTO {table_name} (Name, Age, DOB, Diagnosis, Gender, BloodType, Allergies, LastVisitDate, Medications) VALUES "
    
    # List to hold individual SQL value strings
    sql_values_list = []
    
    for record in records:
        # Assuming fields in each record are comma-separated
        fields = record.split(', ')
        # Ensure the record has the correct number of fields
        if len(fields) == 9:
            # Formatting each field with quotes if it's a string
            formatted_fields = [f"'{field}'" if column_name in ['Name', 'DOB', 'Diagnosis', 'Gender', 'BloodType', 'Allergies', 'LastVisitDate', 'Medications'] else field for field, column_name in zip(fields, columns_format.keys())]
            sql_values_list.append(f"({', '.join(formatted_fields)})")
    
    # Join all VALUES with commas and append to the base INSERT statement
    sql_insert_statements = sql_insert_template + ",\n".join(sql_values_list) + ";"
    
    return sql_insert_statements

# Example synthetic data (replace this with your actual generated data)
generated_data = """John Doe, 30, 1990-01-01, Flu, Male, A+, None, 2023-02-01, Aspirin
Jane Smith, 25, 1995-03-15, Cold, Female, O+, Peanuts, 2023-03-01, Ibuprofen"""

# Assuming 'columns_format' is defined as earlier
table_name = "HealthcareRecords"  # Specify your table name

# Convert to SQL
sql_dump_text = parse_and_convert_to_sql(generated_data, table_name)

print(sql_dump_text)


INSERT INTO HealthcareRecords (Name, Age, DOB, Diagnosis, Gender, BloodType, Allergies, LastVisitDate, Medications) VALUES (John Doe, 30, 1990-01-01, 'Flu', 'Male', 'A+', 'None', '2023-02-01', 'Aspirin'),
(Jane Smith, 25, 1995-03-15, 'Cold', 'Female', 'O+', 'Peanuts', '2023-03-01', 'Ibuprofen');
